# The Effects of Human-Like Reasoning on the Model's Behavior

## Introduction

The HAICOR (Human-AI Collaborative Reasoning) project aims to improve the explainability of machine learning systems for commonsense reasoning about social interactions by facilitating and utilizing human feedback during the model's training and inference process. This experiment reexamines the behavior of the machine learning system proposed by [Paul and Frank (2019)](https://www.aclweb.org/anthology/N19-1368/) and investigates the effects of human-like reasoning. Results produced by this experiment will validate several assumptions held by the porject, identify potential obstacles of utilizing the feedbacks, and indicate promising approaches for future research efforts.

We begin by describing the Story Commonsense dataset and outlining the machine learning systems developed by [Paul and Frank (2019)](https://www.aclweb.org/anthology/N19-1368/).

Story Commonsense, published by [Rashkin et al. (2018)](https://arxiv.org/abs/1805.06533), is a large-scale dataset with low-level annotations that models naive psychology of characters in simple commonsense stories. The dataset contains five-sentence stories that describe a simple scenario between multiple characters. The annotation includes sentence-level information about the character's appearance, the character's actions, the motivations of those actions, and the emotional effects of those actions.

![Figure 1](assets/story-commonsense-example.png)

The figure shows an example of annotation and its story context from the Story Commonsense dataset. The rounded rectangles at the end of each sentence represent the characters that appeared in the whole story context. A grey rectangle indicates that the character is absent from the sentence; a light blue rectangle indicates that the character appears in the sentence but experiences no emotional shift; a blue rectangle indicates that the character appears in the sentence and experiences an emotional shift.

[Paul and Frank (2019)](https://www.aclweb.org/anthology/N19-1368/) approached the Story Commonsense dataset with a supervised deep-learning model enriched using knowledge paths extracted from ConceptNet ([Speer et al., 2016](https://www.aclweb.org/anthology/L12-1639/)). They observed a noticable performance increase when the model is supplied with both the knowledge paths and story context, compared to the story context alone. The model also offers interpretability through the learned attention map over commonsense knowledge paths ([Paul and Frank, 2019](https://www.aclweb.org/anthology/N19-1368/)).

The system consists of two distinct components: a static knowledge path extractor and a supervised deep-learning model. We will provide an overview of their operations. The system requires the current sentence, the current context (the sentences before the current sentence), and the character as inputs; its output is the human needs classification vector. Using the attention mechanism, we can also obtain the attention map over the sentence tokens, context tokens, and knowledge paths.

Given an input, the system begins by generating the ConceptNet knowledge paths. First, path-finding algorithms generates all shortest paths between concepts from ConceptNet that appeared either in the sentence or context, and human need concepts. Second, several standard graph centrality metrics are calculated for each path. Third, all paths are ranked using the centrality metrics.

Top paths are selected and stored as set (order-insensitive) with ranking information discarded; the exact number of selection size is a hyperparameter of the system. The deep-learning model takes the sentence, context, character, and knowledge paths as inputs to produce a human needs classification. The problem is framed as a multi-label classification problem and optimized using Adam with cross-entropy as the cost function.

![Figure 2](assets/pauls-model-diagram.png)

The illustration shows an architectural overview of Paul's model. Each rectangle represents a component of the system; each arrow represents information passed between components. The model executes from top-left to bottom-right.

## Previous Works

The HAICOR project has spent considerable efforts on patching the source code, provided by the original author, to a partially executable state. At the time of the experiment, the static knowledge path extractor is not operational; however, its primary function of knowledge path generation is currently substituted by an extensive collection of ConceptNet knowledge paths, provided by the original author. The experiment scripts and the best-performing model have been patched to an executable state and updated using [automated tools](https://www.tensorflow.org/guide/upgrade) to Tensorflow 2. Given the source code's fragility, all interactions with the model are indirectly performed by the experiment script.

The team has conducted one preliminary experiment on the effects of knowledge paths. However, the experiment results revealed an implementation error that is later fixed. In light of the issue, the original author also provides a clean implementation of the best-performing model; however, at the time of the experiment, it has not been patched to an operational state. The experiment results also revealed a severe lack of understanding of the system's behavior among the team members, which partially motivated this experiment.

An internal pilot study that focuses on the effects of human-modified reasoning on the model's behavior has been conducted. The results suggest a negligible performance increase could be gained, and most subjects experience difficulty during the modification process. There are two significant differences between the pilot study and this experiment; the pilot study constrains human subjects to modifications of existing knowledge paths, while this study does not have such restriction; the pilot study only investigated a single inference instance, while this study investigated several instances.

## Experiment

The experiment's primary objective was to investigate the effects of human-like reasoning on the system's behavior. However, given that only the deep-learning model is functional at the time of the experiment, the experiment's primary object became the investigation of the effects of human-like reasoning on the model's behavior. The experiment's auxiliary goal is to investigate several assumptions about the model not investigated by [Paul and Frank (2019)](https://www.aclweb.org/anthology/N19-1368/).

In this experiment, we consider human-like reasoning from two aspects inspired by linguistic: syntax and semantic. The deep-learning model handles the knowledge paths as a set (order-insensitive) of template-based natural language sentences, as illustrated below. The example sentence does not possess a human-like syntax, meaning the sentence is unlikely to be produced by a human, nor human-like semantic, meaning the sentence context is unlikely to be produced by a human.

`morning related to section related to family`

An example of the template-based natural language representation of a knowledge path.

[Shwartz et al. (2020)](https://arxiv.org/abs/2004.05483) proposed an unsupervised framework based on self-talk as a novel alternative to multiple-choice commonsense tasks. Utilizing pre-trained language models and ConceptNet, their work is able to generate natural language representation of ConceptNet path. Although lacking in semantic, the generated sentences are human-like in terms of syntax (example below).

`In order for cook to happen, wear an apron needs to happen.`

An example of the self-talk-based natural language representation of a knowledge path.

Two sets of reasoning paths are collected from two independent investigations prior to the experiment. The investigation of the work of [Shwartz et al. (2020)](https://arxiv.org/abs/2004.05483) resulted in 3 reasoning paths been generated, using their system, for 16 instances of the Story Commonsense dataset; the investigation conducted by one of the group members resulted in 3 reasoning paths, manually created, for 7 of the Story Commonsense dataset (example below).

`Gary wanted to use a website. The person shows acceptance to it. Approval.`

An example of the human-generated natural language representation of a knowledge path.

Given the insufficient amount of reasoning paths, it is impossible to retrain the deep-learning model to observe any behavior difference; therefore, the experiment is performed using a pre-trained model, achieving an $F_1$ score of 31.84%. The performance is noticeably worse than the performance reported by Paul and Frank; this is likely due to the smaller training set caused by the inoperative knowledge path extractor. Therefore, all the behavior changes will be measured relative to the baseline performance of the used model.

|                     |   Unnatural Syntax   |     Natural Syntax    |
|:-------------------:|:--------------------:|:---------------------:|
| Irrational Semantic | Template-based paths | Self-talk-based paths |
|  Rational Semantic  |                      | Human-generated paths |

The table categorizes the three types of knowledge paths utilized by the experiment.

The experiment has three independent variables - the inference instance, which is differentiated based on the story, sentence, and character; the number of human-like knowledge paths, which ranges from 0 to 3; the number of system-generated paths, which ranges from 0 to the total number of available system-generated paths. The exact knowledge paths selection, and order, was randomized using a uniform distribution. The dependent variable of the experiment consists of the model's prediction vector, knowledge path attention vector, and cross-entropy error measured against golden labels.

An experiment was conducted for each inference to investigate the effect of the remaining two independent variables on the dependent variable. One hundred inferences, refer to as samples, were generated for each possible configuration of the number of human-like and system-generated knoweldge paths, refer to as an instance. This procedure was used to obtain the experimental data for self-talk-based and human-generated knowledge paths; no sample contains both self-talk based and human-generated knowledge paths.

## Research Questions

This section contains links to all the research questions investigated by this experiment.

1. [Order-Invariant Assumption](Research%20Question%201.ipynb).
2. [Attention Weight and Path Contribution](Research%20Question%202.ipynb).
3. [Attention Weight and Path Type](Research%20Question%203.ipynb).